In [1]:
import sys
import json

catchment_radius={'bus':'500','subway':'800','rail':'1000'}
footpaths = {'max_length':'1000', 'speed':'2.8','n_ntlegs':'2'}
use_road_netowrk = True
params = {'catchment_radius':catchment_radius,'footpaths':footpaths, 'use_road_network': use_road_netowrk}

default = {'training_folder': '../../scenarios/clermont', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)

{'training_folder': '../../scenarios/clermont', 'params': {'catchment_radius': {'bus': '500', 'subway': '800', 'rail': '1000'}, 'footpaths': {'max_length': '1000', 'speed': '2.8', 'n_ntlegs': '2'}, 'use_road_network': True}}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numpy as np
import random
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from syspy.spatial.spatial import add_geometry_coordinates, nearest
from sklearn.neighbors import NearestNeighbors
from typing import Literal
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel

#num_cores = 1
print('numba threads',nb.config.NUMBA_NUM_THREADS)

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio'

numba threads 20


In [3]:
def get_epsg(lat: float, lon: float) -> int:
    '''
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

# Folders stucture and params

everything is on S3 (nothing on ECR) so no direct input folder. just scenarios/{scen}/inputs/

In [4]:
use_road_network = argv['params'].get('use_road_network')

In [5]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')


In [6]:
if 'footpaths' in argv['params'].keys():
    max_length = float(argv['params' ]['footpaths']['max_length'])
    speed = float(argv['params' ]['footpaths']['speed'])
    n_ntlegs = int(argv['params' ]['footpaths']['n_ntlegs'])
else:
    max_length = float(footpaths['max_length'])
    speed = float(footpaths['speed'])
    n_ntlegs = int(footpaths['n_ntlegs'])

In [ ]:
end_of_notebook

# Inputs

In [7]:
links = gpd.read_file(pt_folder + 'links.geojson',engine=io_engine) 
nodes = gpd.read_file(pt_folder + 'nodes.geojson',engine=io_engine)
links = links.set_index('index')
nodes = nodes.set_index('index')

Skipping field road_link_list: unsupported OGR type: 5


In [8]:
rnodes_file = os.path.join(road_folder, 'road_nodes.geojson')
rnodes_file_provided = os.path.isfile(rnodes_file)
if rnodes_file_provided:
    rnodes = gpd.read_file(os.path.join(road_folder, 'road_nodes.geojson'), engine=io_engine)
    rnodes = rnodes.set_index('index')
    rlinks = gpd.read_file(os.path.join(road_folder, 'road_links.geojson'), engine=io_engine)
    rlinks = rlinks.set_index('index')
print('rnodes?',rnodes_file_provided)

rnodes? True


In [9]:
od_file = os.path.join(od_folder, 'od.geojson')
od_file_provided = os.path.isfile(od_file)

if od_file_provided:
    od_test = gpd.read_file(od_file, engine=io_engine)
    if 'index' not in od_test.columns:
        od_test.reset_index(names='index', inplace=True)
    if 'name' not in od_test.columns:
        od_test['name'] = od_test['index'].astype(str)
    od_test['name'] = od_test['name'].fillna(od_test['index'].astype(str))

    od_test = od_test[['name', 'origin', 'destination', 'volume', 'geometry']]
else:
    print('end of pathfinder')
    end_of_notebook

end_of_notebook here if OD_file not provided!!

In [10]:
od_test['volume'] = od_test['volume'].map(int).astype(float)

In [12]:
od_test['geometry_o'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][0]))
od_test['geometry_d'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][1]))

if 'origin' not in od_test.columns:
    od_test['origin'] = od_test['index'].astype(str) + '_o' 
od_test['origin'] = od_test['origin'].astype(str)

if 'destination' not in od_test.columns:
    od_test['destination'] = od_test['index'].astype(str) + '_d' 
od_test['destination'] = od_test['destination'].astype(str)

zones = od_test.copy()
zones_d = od_test.copy()
zones['geometry'] = zones['geometry_o']
zones_d['geometry'] = zones_d['geometry_d']
zones['index'] = zones['origin'].astype(str)
zones_d['index'] = zones_d['destination'].astype(str)
zones = pd.concat([zones[['index','geometry']],zones_d[['index','geometry']]]).drop_duplicates()
zones = zones.set_index('index')

od_set = set(zip(od_test['origin'], od_test['destination']))

# preparation

In [13]:
def add_col(links,col='length'):
    if col not in links.columns:
        return True
    elif any(links[col].isnull()):
        return True
    else:
        return False

In [14]:
if add_col(links,'length'):
    print('add length')
    centroid = [*LineString(nodes.centroid.values).centroid.coords][0]
    crs = get_epsg(centroid[1],centroid[0])
    print('create length from geometry')
    links['length'] = links.to_crs(crs).length
    
if add_col(links,'speed'):
    print('add speed')
    links['speed'] = links['length']/links['time'] * 3.6

# Walkmodel

In [15]:
sm = stepmodel.StepModel(epsg=4326)
sm.links = links
sm.nodes = nodes
sm.zones = zones

if use_road_network:
    sm.road_links = rlinks
    sm.road_links['walk_time'] = sm.road_links['length'] / (3000/3600)
    
    sm.road_nodes = rnodes

In [16]:
# We remove highway for the walkmodel
#walk_type = ['Artere','Collectrices','Locale','Nationale','Regionale']
#walkmodel.road_links = walkmodel.road_links[~walkmodel.road_links['highway'].isin(walk_type)]
#walkmodel.road_nodes = walkmodel.road_nodes.loc[list(np.unique(walkmodel.road_links[['a','b']].values.flatten()))]
#walkmodel.preparation_clusterize_nodes(distance_threshold=0.000002)
# Footpath between PT nodes
sm.preparation_footpaths(speed=speed, max_length=max_length, clusters_distance=0.00001)

In [17]:
# Access footpaths (zone_to_road and road_to_transit)
sm.preparation_ntlegs(
    short_leg_speed=speed,
    long_leg_speed=speed, # tout le monde marche
    threshold=1000,
    zone_to_transit=False,
    zone_to_road=True,
    road_to_transit=True,
    n_ntlegs=n_ntlegs,
    max_ntleg_length=200000,
)

In [18]:
# Copy connectors to the model
sm.footpaths = sm.footpaths.drop(columns='voronoi')
#sm.footpaths = walkmodel.footpaths.drop(columns='voronoi').copy()
#sm.zone_to_transit = walkmodel.zone_to_transit.copy()

In [19]:
sm.footpaths['speed'] = sm.footpaths['length']/sm.footpaths['time'] * 3.6

# utils

# pathfinder

test test test

In [20]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set
)

start publicpathfinder
898 sources 898 targets direct search
path_analysis


In [21]:
sm.pt_los

,origin,destination,gtime,path,pathfinder_session,reversed
0,0,0,0.000000,"(0,)",best_path,False
1,0,1,33779.813010,"(0, rnode_124533, rnode_56866, rnode_4193, rno...",best_path,False
2,0,10,68143.018358,"(0, rnode_124533, rnode_56866, rnode_4193, rno...",best_path,False
3,0,100,34503.828562,"(0, rnode_124533, rnode_56866, rnode_4193, rno...",best_path,False
4,0,101,71214.089848,"(0, rnode_124533, rnode_56866, rnode_4193, rno...",best_path,False
...,...,...,...,...,...,...
443325,99,9,120449.488350,"(99, rnode_35056, rnode_16578, rnode_129368, r...",best_path,False
443326,99,90,118410.049281,"(99, rnode_35056, rnode_16578, rnode_129368, r...",best_path,False
443327,99,92,60352.362852,"(99, rnode_35056, rnode_16578, rnode_129368, r...",best_path,False
443328,99,93,121668.769598,"(99, rnode_35056, rnode_16578, rnode_129368, r...",best_path,False


In [22]:
def extract_rlink_path(path, rlinks):
    links_path_set = {f"{path[i]}__{path[i+1]}" for i in range(len(path) - 1)}
    res = rlinks.loc[rlinks['ab_id'].isin(links_path_set), 'index'].values.tolist()
    return res

In [23]:
rlinks = sm.road_links[['a', 'b', 'geometry']].copy().reset_index(names='index')
rlinks['ab_id'] = rlinks['a'] + '__' + rlinks['b']

paths = sm.pt_los[sm.pt_los['gtime'] != 0.]['path'].values.tolist()
for path in paths:
    g = extract_rlink_path(path, rlinks)

KeyboardInterrupt: 

In [ ]:
sm.analysis_pt_los(walk_on_road=True)

path_analysis: 100%|██████████| 440689/440689 [00:51<00:00, 8527.88it/s] 


ValueError: cannot reindex on an axis with duplicate labels

# create path

In [ ]:
od_links = od_test.merge(sm.pt_los[['origin','destination','gtime','link_path']],on=['origin','destination'])
od_links = od_links.drop(columns = ['geometry','geometry_o','geometry_d','origin','destination'])
od_links = od_links.explode('link_path')
# may add route_width if quenedi can handle it one day.
od_links = od_links.merge(sm.links[['route_color','geometry','time','speed']],left_on='link_path',right_index=True)
od_links = od_links.drop(columns='link_path')

In [ ]:
od_ntlegs = od_test.merge(sm.pt_los[['origin','destination','gtime','ntlegs']],on=['origin','destination'])
od_ntlegs = od_ntlegs.drop(columns = ['geometry','geometry_o','geometry_d','origin','destination'])
od_ntlegs = od_ntlegs.explode('ntlegs')

if use_road_network:
    ntlegs_dict1 = sm.zone_to_road.reset_index().set_index(['a','b'])['index'].to_dict()
    ntlegs_dict2 = sm.road_to_transit.reset_index().set_index(['a','b'])['index'].to_dict()
od_ntlegs['ntlegs'] = od_ntlegs['ntlegs'].apply(ntlegs_dict.get)

od_ntlegs = od_ntlegs.merge(sm.zone_to_transit[['geometry','time','speed']],left_on='ntlegs',right_index=True)
od_ntlegs = od_ntlegs.drop(columns='ntlegs')
#od_ntlegs['route_color']='4B4B4B'

AttributeError: 'StepModel' object has no attribute 'zone_to_transit'

In [ ]:
od_footpaths = od_test.merge(sm.pt_los[['origin','destination','gtime','footpaths']],on=['origin','destination'])
od_footpaths = od_footpaths.drop(columns = ['geometry','geometry_o','geometry_d','origin','destination'])
od_footpaths = od_footpaths.explode('footpaths')

footpaths_dict = sm.footpaths.reset_index().set_index(['a','b'])['index'].to_dict()
od_footpaths['footpaths'] = od_footpaths['footpaths'].apply(footpaths_dict.get)

od_footpaths = od_footpaths.merge(sm.footpaths[['geometry','time','speed']],left_on='footpaths',right_index=True)
od_footpaths = od_footpaths.drop(columns='footpaths')

In [ ]:
od_route = pd.concat([od_links,od_footpaths,od_ntlegs],axis=0)

In [ ]:
od_route['route_color'] = od_route['route_color'].fillna('838383')
od_route['route_color'] = '#' + od_route['route_color']

In [ ]:
od_route = od_route.rename(columns={'name':'od_name'}).drop(columns='index')
od_route.reset_index(drop=True)
od_route.index.name='index'

In [ ]:
od_route = gpd.GeoDataFrame(od_route,crs=4326)
od_route.to_file(output_folder+'od_route.geojson',driver='GeoJSON',engine=io_engine)

# Formation :
Exporter des metrics (du pt_los) sur les OD trouvés en format csv <br>
 (ex: time, mode, etc). garder la colonne "name" s.v.p



In [ ]:
sm.analysis_pt_route_type(['rail','subway','bus'])
pt_los = od_test[['origin','destination','name']].merge(sm.pt_los,on=['origin','destination'])

In [ ]:
pt_los

,origin,destination,name,gtime,path,pathfinder_session,reversed,boardings,alightings,node_path,...,ntlegs,transfers,boarding_links,alighting_links,all_walk,ntransfers,time_link_path,length_link_path,route_types,route_type
0,OD_29jjnnWWXWDoZKHfXqtSJx_o,OD_29jjnnWWXWDoZKHfXqtSJx_d,sb-systra,2454.382414,"(OD_29jjnnWWXWDoZKHfXqtSJx_o, node_3426, node_...",best_path,False,[node_8879],[node_200],"[node_3426, node_3427, node_3393, node_536, no...",...,"[(OD_29jjnnWWXWDoZKHfXqtSJx_o, node_3426), (no...",[],[link_13686],[link_13694],False,0,720,6475.076788,"(subway,)",subway
1,OD_kykfkAXaKsTygo9fDsWMFG_o,OD_kykfkAXaKsTygo9fDsWMFG_d,systra_sb,2128.517417,"(OD_kykfkAXaKsTygo9fDsWMFG_o, node_14159, node...",best_path,False,"[node_8861, node_206, node_10487]","[node_14345, node_220, node_10505]","[node_14159, node_2943, node_4598, node_258, n...",...,"[(OD_kykfkAXaKsTygo9fDsWMFG_o, node_14159), (n...",[],"[link_13749, link_13668, link_9230]","[link_13752, link_13673, link_9231]",False,2,904,7384.129403,"(subway, bus)",subway
2,OD_79HUhzRX6D5HBwNSgVwSV2_o,OD_79HUhzRX6D5HBwNSgVwSV2_d,sb-jb,3191.265807,"(OD_79HUhzRX6D5HBwNSgVwSV2_o, node_3426, node_...",best_path,False,"[node_8879, node_8888, node_14345, node_14341]","[node_14342, node_206, node_14340, node_14009]","[node_3426, node_3427, node_3393, node_536, no...",...,"[(OD_79HUhzRX6D5HBwNSgVwSV2_o, node_3426), (no...",[],"[link_13686, link_13667, link_13753, link_13827]","[link_13691, link_13667, link_13758, link_13827]",False,3,1200,11192.828333,"(subway,)",subway
3,OD_eKAH5Dq14xF92bAh2Ned9y_o,OD_eKAH5Dq14xF92bAh2Ned9y_d,jb-sb,2590.451913,"(OD_eKAH5Dq14xF92bAh2Ned9y_o, node_1792, node_...",best_path,False,"[node_310, node_14339, node_206, node_10487]","[node_278, node_14344, node_220, node_10505]","[node_1792, node_309, node_310, node_278, node...",...,"[(OD_eKAH5Dq14xF92bAh2Ned9y_o, node_1792), (no...",[],"[link_13842, link_13713, link_13668, link_9230]","[link_13842, link_13718, link_13673, link_9231]",False,3,1144,10677.531006,"(subway, bus)",subway
4,OD_sDYb2uoJCn9iznErFzKb36_o,OD_sDYb2uoJCn9iznErFzKb36_d,sf-systra,2451.401571,"(OD_sDYb2uoJCn9iznErFzKb36_o, node_2474, link_...",best_path,False,"[node_2474, node_2582, node_14333]","[node_2583, node_13899, node_259]","[node_2474, node_2583, node_2582, node_13899, ...",...,"[(OD_sDYb2uoJCn9iznErFzKb36_o, node_2474), (no...",[],"[link_9715, link_7358, link_13716]","[link_9716, link_7363, link_13722]",False,2,1009,6160.051753,"(subway, bus)",subway
5,OD_rgnwdTHKRA6H5kNMyRBgfS_o,OD_rgnwdTHKRA6H5kNMyRBgfS_d,systra-sf,2601.883487,"(OD_rgnwdTHKRA6H5kNMyRBgfS_o, node_2864, node_...",best_path,False,"[node_8891, node_10446]","[node_212, node_2472]","[node_2864, node_2883, node_10085, node_10086,...",...,"[(OD_rgnwdTHKRA6H5kNMyRBgfS_o, node_2864), (no...",[],"[link_13664, link_9646]","[link_13669, link_9657]",False,1,1187,6862.255227,"(subway, bus)",subway
6,OD_bQtsoSivwVQME4XNKwr2vM_o,OD_bQtsoSivwVQME4XNKwr2vM_d,sb-sf,2885.972033,"(OD_bQtsoSivwVQME4XNKwr2vM_o, node_10496, link...",best_path,False,"[node_10496, node_2978]","[node_623, node_9987]","[node_10496, node_623, node_691, node_2978, no...",...,"[(OD_bQtsoSivwVQME4XNKwr2vM_o, node_10496), (n...",[],"[link_9243, link_9025]","[link_9245, link_9033]",False,1,1121,4315.300977,"(bus,)",bus
7,OD_icVzMR6FonccfVTQzMtj5i_o,OD_icVzMR6FonccfVTQzMtj5i_d,sf-sb,2563.224628,"(OD_icVzMR6FonccfVTQzMtj5i_o, node_9932, node_...",best_path,False,"[node_2474, node_212, node_10487]","[node_3318, node_220, node_10505]","[node_9932, node_2474, node_3318, node_10446, ...",...,"[(OD_icVzMR6FonccfVTQzMtj5i_o, node_9932), (no...",[],"[link_9715, link_13670, link_9230]","[link_9725, link_13673, link_9231]",False,2,1233,6752.881711,"(subway, bus)",subway
8,OD_dRz4xxHQNaSEo2XzDkLUfi_o,OD_dRz4xxHQNaSEo2XzDkLUfi_d,trip,4724.146896,"(OD_dRz4xxHQNaSEo2XzDkLUfi_o, node_3426, node_...",best_path,False,"[node_220, node_3999]","[node_232, node_4446]","[node_3426, node_3427, node_3393, node_536, no...",...,"[(OD_dRz4xxHQNaSEo2Xz

In [ ]:
print('pathfinder successfull')

pathfinder successfull


# test